In [1]:
# Importamos librerias
import cv2
import os
import mediapipe as mp
import imutils

In [5]:
personName = 'Uriel' # Nombre de la persona que se analizara su video 
dataPath = 'C:/Users/uriel/OneDrive - Universidad de Guanajuato/Documentos/8vo semestre/Forte/Rostros'# Lugar donde se van a almacenar los rostros
personPath = dataPath + '/' + personName 

if not os.path.exists(personPath): # Verifica si existe la carpete de la persona si no crea una
    print('Carpeta creada: ',personPath)
    os.makedirs(personPath)

Carpeta creada:  C:/Users/uriel/OneDrive - Universidad de Guanajuato/Documentos/8vo semestre/Forte/Mariel


In [6]:
mp_face_detection = mp.solutions.face_detection # Es el detector de rostros
mp_drawing = mp.solutions.drawing_utils # Esto es para colocar el recuadro y los puntos
cap = cv2.VideoCapture('video_u.mp4') # Configuramos que tomaremos video
count = 0 # Esta variable va a llevar la cuenta de las fotos que se han tomado 

In [7]:
with mp_face_detection.FaceDetection(  # Gestor de contexto
    min_detection_confidence=0.6) as face_detection:
    while True:
        ret, frame = cap.read() # Comenzamos a leer el video
        if ret == False: # Verificamos si se esta leyendo de forma correcta
            break
        frame = imutils.resize(frame, width=640)
        height, width, _ = frame.shape # Obtenemos las dimensipones de la imagen
        aux_frame = frame.copy() # Copiamos el frame 
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # cv2 lo lee como BGR y lo convertimos a RGB       
        results = face_detection.process(frame_rgb) # Obtenemos las cordenadas de la cara y de los puntos de interes
        k = cv2.waitKey(1) & 0xFF
        if results.detections is not None: # Si detecta un rostro entra aqui 
            for detection in results.detections: # Se usa un for en caso de detectar mas de un rostro
                mp_drawing.draw_detection(frame, detection,
                    mp_drawing.DrawingSpec(color=(0, 255, 255), circle_radius=2),
                    mp_drawing.DrawingSpec(color=(255, 0, 255))) # Dibuja en un recuadro un rostro y con circulos los puntos de interes
                
                # Obtenemos la posicion de la cara
                x = int(detection.location_data.relative_bounding_box.xmin * width)
                y = int(detection.location_data.relative_bounding_box.ymin * height)
                w = int(detection.location_data.relative_bounding_box.width * width)
                h = int(detection.location_data.relative_bounding_box.height * height)
                
                # Recortamos el rostro y reajustamos el tamaño
                rostro = aux_frame[y:y+h,x:x+w]
                rostro = cv2.resize(rostro,(150,150), interpolation=cv2.INTER_CUBIC)
                cv2.imwrite(personPath + '/rotro_{}.jpg'.format(count),rostro)
                cv2.imshow('rostro',rostro) # Mostramos el rostro guardado 
                count = count +1 
                
        cv2.imshow("frame", frame) # Muestra lo que esta detectando 
        if k == 27 or count >= 300: # Si tecleamos la tecla esc se cancela o si llega a 300
            break
cap.release()
cv2.destroyAllWindows()